In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import *

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/21 16:40:00 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.25.28.45 instead (on interface eth0)
22/08/21 16:40:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/21 16:40:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
import pandas as pd

In [2]:
yellow = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_credit')

In [3]:
yellow = yellow.withColumn(
    'Win',
    (F.col("Win") == 'Yes').cast('BOOLEAN')
)

yellow.show(1, vertical=True, truncate=100)

-RECORD 0------------------------------------
 tpep_pickup_datetime  | 2018-12-25 00:39:10 
 tpep_dropoff_datetime | 2018-12-25 01:04:33 
 trip_distance         | 2.56                
 PULocationID          | 162                 
 DOLocationID          | 263                 
 fare_amount           | 16.5                
 tip_amount            | 4.58                
 total_amount          | 22.88               
 day_of_week           | Tue                 
 is_weekend            | false               
 trip_length           | 25.3833             
 Date                  | 2018-12-25          
 Start(ET)             | 12:00               
 Attendance            | 19812               
 Win                   | false               
 margin_victory/loss   | -14                 
only showing top 1 row



In [4]:
yellow.groupBy('Date').count().orderBy('Date')


Date,count
2018-10-17,182129
2018-10-20,169804
2018-10-26,181843
2018-10-29,156363
2018-10-31,164490
2018-11-05,161472
2018-11-11,148113
2018-11-20,166765
2018-11-23,99489
2018-12-01,180716


In [5]:
# converts the pickup_hours into categorical data as there are correlation that could be seen from analysis

# drop the tpep pickup and dropoff columns

yellow = yellow.withColumn('pickup_hour', hour("tpep_pickup_datetime"))
yellow = yellow.withColumn('dropoff_hour', hour("tpep_dropoff_datetime"))

yellow = yellow.drop('tpep_pickup_datetime', 'tpep_dropoff_datetime')

In [6]:
# pick 2018-11-20 as train set after randomly picking it manually
yellow_train = yellow.where((F.col('Date') == '2018-11-20'))

# pick 2018-11-23 as the test set as it is the next game date after 2018-11-20
yellow_test = yellow.where((F.col('Date') == '2018-11-23'))



In [7]:
yellow_train = yellow_train.drop('Date','Start(ET)')

In [8]:
yellow_test = yellow_test.drop('Date', 'Start(ET)')

In [9]:
yellow_train

trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,total_amount,day_of_week,is_weekend,trip_length,Attendance,Win,margin_victory/loss,pickup_hour,dropoff_hour
0.88,79,249,6.5,1.82,9.12,Tue,false,6.8333,19812,false,-4,11,11
1.79,90,211,10.0,2.16,12.96,Tue,false,13.3333,19812,false,-4,13,13
1.36,211,90,7.0,1.0,8.8,Tue,false,7.8667,19812,false,-4,13,13
1.48,100,170,17.5,0.0,18.3,Tue,false,35.6,19812,false,-4,14,14
0.78,170,234,6.0,2.04,8.84,Tue,false,6.6333,19812,false,-4,14,14
2.19,234,246,15.0,2.0,17.8,Tue,false,22.9167,19812,false,-4,14,15
1.84,158,144,13.5,2.86,17.16,Tue,false,20.1,19812,false,-4,15,15
2.65,140,239,13.5,2.5,16.8,Tue,false,18.65,19812,false,-4,19,19
2.16,239,41,11.5,1.0,13.3,Tue,false,15.6833,19812,false,-4,19,19
1.04,41,74,5.5,1.26,7.56,Tue,false,4.9333,19812,false,-4,19,19


In [16]:
from pyspark.ml.feature import VectorAssembler

features = 'features'

input_cols = ['fare_amount', 'trip_distance', 'total_amount', 'is_weekend', 'trip_length', 'Attendance', 'Win', 'margin_victory/loss',
             'PULocationID', 'DOLocationID', 'pickup_hour', 'dropoff_hour']

vectorAssembler = VectorAssembler(
    inputCols = input_cols, 
    outputCol = features)

v_yellow_train = vectorAssembler.transform(yellow_train.dropna('any'))

v_yellow_train = v_yellow_train.select(['features', 'tip_amount'])

v_yellow_train.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[6.5,0.88,9.12,0....|      1.82|
|[10.0,1.79,12.96,...|      2.16|
|[7.0,1.36,8.8,0.0...|       1.0|
+--------------------+----------+
only showing top 3 rows



In [17]:
from pyspark.ml.regression import LinearRegression
lm = LinearRegression(
    featuresCol='features', 
    labelCol='tip_amount'
).fit(v_yellow_train)


22/08/21 16:44:33 WARN Instrumentation: [31a17585] regParam is zero, which might cause numerical instability and overfitting.


22/08/21 16:44:35 WARN Instrumentation: [31a17585] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [18]:
# Access coefficients
pd.DataFrame(
    data=[lm.intercept] + list(lm.coefficients),
    index=['intercept'] + input_cols,
    columns=['coefficient']
)

,coefficient
intercept,-0.472782
fare_amount,-0.712973
trip_distance,-0.000818
total_amount,0.687725
is_weekend,0.000000
trip_length,0.043450
Attendance,0.000000
Win,0.000000
margin_victory/loss,0.000000
PULocationID,-0.000054
